In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import json
import sys
import os
import requests
# setting path
sys.path.append('../common')
import utils
import limitUtils
import fillInUtils

SETUP_PATH = 'setup.json'
f = open(SETUP_PATH)
data = json.load(f)

### Fill NestJS

In [20]:
response = requests.get(data['app']["url"])
content = response.content.decode("utf-8")
len(json.loads(content))

100000

In [8]:
fillInUtils.fill_framework(
    app=data['app'],
    N = data['fill_in_quantity'],
)

http://localhost:3000/member/clear/
b''
201
http://localhost:3000/member/fill-in/
100000
iterations: 100
1/100: b'Ok'
1/100: 201
1/100: duration: 0s
1/100: expected end in: 0s
2/100: b'Ok'
2/100: 201
2/100: duration: 0s
2/100: expected end in: 0s
3/100: b'Ok'
3/100: 201
3/100: duration: 0s
3/100: expected end in: 0s
4/100: b'Ok'
4/100: 201
4/100: duration: 0s
4/100: expected end in: 0s
5/100: b'Ok'
5/100: 201
5/100: duration: 0s
5/100: expected end in: 0s
6/100: b'Ok'
6/100: 201
6/100: duration: 0s
6/100: expected end in: 0s
7/100: b'Ok'
7/100: 201
7/100: duration: 0s
7/100: expected end in: 0s
8/100: b'Ok'
8/100: 201
8/100: duration: 0s
8/100: expected end in: 0s
9/100: b'Ok'
9/100: 201
9/100: duration: 0s
9/100: expected end in: 0s
10/100: b'Ok'
10/100: 201
10/100: duration: 0s
10/100: expected end in: 0s
11/100: b'Ok'
11/100: 201
11/100: duration: 0s
11/100: expected end in: 0s
12/100: b'Ok'
12/100: 201
12/100: duration: 0s
12/100: expected end in: 0s
13/100: b'Ok'
13/100: 201
13/10

## Run k6 test script

In [22]:
limitUtils.test_vus_limit(
    result_base_dir=data["test_result_base_dir"],
    test_result_dir_prefix=data["test_result_dir_prefix"],
    script_path=data["script_path"],
    setup_path=SETUP_PATH,
    app=data["app"],
    inital_vus=163,
    out_file_name="out_fill_in.txt",
    end=164
)

date and time = 2024-06-01_17-51-44
./out/out_2024-06-01_17-51-44

          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: ../../grafana-k6/script.js
        output: json (./out/out_2024-06-01_17-51-44/nestjs.json), csv (./out/out_2024-06-01_17-51-44/nestjs.csv)

     scenarios: (100.00%) 1 scenario, 163 max VUs, 1m0s max duration (incl. graceful stop):
              * default: 163 looping VUs for 30s (gracefulStop: 30s)


running (0m00.9s), 163/163 VUs, 0 complete and 0 interrupted iterations
default   [   3% ] 163 VUs  00.9s/30s

running (0m01.9s), 163/163 VUs, 0 complete and 0 interrupted iterations
default   [   6% ] 163 VUs  01.9s/30s

running (0m02.9s), 163/163 VUs, 0 complete and 0 interrupted iterations
default   [  10% ] 163 VUs  02.9s/30s

running (0m03.9s), 163/163 VUs, 0 complete and 0 interrupted iter

time="2024-06-01T17:52:46+02:00" level=warning msg="No script iterations fully finished, consider making the test duration longer"


Incorrect part for vus = 163: nan



/home/jan/Dokumenty/Studia/MSem3/Magisterka/sandbox/framework-benchmark/tests/04-limit-nestjs/../common/utils.py:23: RuntimeWarning: invalid value encountered in scalar divide
  incorrect_part = incorrect_status.count() / (correct_status.count() + incorrect_status.count())
